In [1]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

'\nA Multilayer Perceptron implementation example using TensorFlow library.\nThis example is using the MNIST database of handwritten digits\n(http://yann.lecun.com/exdb/mnist/)\n\nAuthor: Aymeric Damien\nProject: https://github.com/aymericdamien/TensorFlow-Examples/\n'

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
sess = tf.InteractiveSession()
sess.close()

In [15]:
entrada = np.array([[1,2,3], [2, 4, 6], [3, 6, 9], [4, 8, 12]])
salida = [1, 2, 3, 4]
print(entrada)
total_examples = entrada.shape[0]
# Normalizando
mean = np.mean(entrada, axis = 0)
entrada = entrada - media
print(mean)
print(entrada)
std2 = np.mean(entrada**2, axis = 0)
print(std2)
entrada/= std2
print(entrada)

[[ 1  2  3]
 [ 2  4  6]
 [ 3  6  9]
 [ 4  8 12]]
[ 2.5  5.   7.5]
[[-1.5 -3.  -4.5]
 [-0.5 -1.  -1.5]
 [ 0.5  1.   1.5]
 [ 1.5  3.   4.5]]
[  1.25   5.    11.25]
[[-1.2        -0.6        -0.4       ]
 [-0.4        -0.2        -0.13333333]
 [ 0.4         0.2         0.13333333]
 [ 1.2         0.6         0.4       ]]


In [25]:
entrada = np.genfromtxt('C:\\Users\\hp\\Documents\\TFG\\Notebooks\\entrada.csv', delimiter=',').T
total_examples = entrada.shape[0]
# Normalizando
mean = np.mean(entrada, axis = 0)
entrada = entrada - mean
std2 = np.mean(entrada**2, axis = 0)
entrada/= std2

salida = np.genfromtxt('C:\\Users\\hp\\Documents\\TFG\\Notebooks\\salidafac43.csv', delimiter = ',')
# 1hot encoding
salida = pd.get_dummies(list(salida)).values

print(entrada.shape)
print(salida.shape)


m_train = int(total_examples * 0.75)
m_test = total_examples - m_train
X_train = entrada[0:(m_train),:]
Y_train = salida[0:(m_train), :]
X_test = entrada[m_train:total_examples,:]
Y_test = salida[m_train:total_examples, :]

from sklearn.utils import shuffle
X_train, Y_train = shuffle(X_train, Y_train, random_state=10)
print(X_train)
print(Y_train)

(46657, 132)
(46657, 2)
[[ 0.03205161 -0.13617751  0.0358077  ..., -0.15070282  0.04516304
  -0.15473172]
 [ 0.00125132 -0.01384508  0.00230095 ..., -0.00330612  0.01596701
  -0.00418618]
 [-0.00086877  0.12326346 -0.00021004 ...,  0.08616855  0.00336041
   0.09433222]
 ..., 
 [ 0.0144234   0.0242979   0.0147555  ...,  0.03101431  0.02119768
   0.03523143]
 [-0.00855412 -0.04962559 -0.00680894 ..., -0.0474899   0.00606121
  -0.04585915]
 [ 0.01952733  0.02416904  0.01850191 ..., -0.0010919   0.00775349
  -0.00351982]]
[[1 0]
 [1 0]
 [1 0]
 ..., 
 [0 1]
 [1 0]
 [1 0]]


In [19]:
print(np.sum(Y_train, axis = 0))
print(np.sum(Y_test, axis = 0))

[33437  1555]
[10856   809]


In [20]:
# Parameters
learning_rate = 0.001
training_epochs = 50
batch_size = 10000
display_step = 4

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 132 # 66 * 2 features
n_classes = 2  # abierto o cerrado

# tf Graph input
x = tf.placeholder("float32", [None, n_input])
y = tf.placeholder("float32", [None, n_classes])

In [21]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [22]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [23]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(m_train/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = X_train[i * batch_size: ((i+1)*batch_size),]
            batch_y = Y_train[i * batch_size: ((i+1)*batch_size),]
            #batch_y = batch_y.reshape((-1,1))
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    #Y_test = Y_test.reshape((-1,1))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))
    predict = tf.argmax(pred, 1)
    predd = predict.eval({x: X_test})

Epoch: 0001 cost= 3.588157018
Epoch: 0005 cost= 1.240242561
Epoch: 0009 cost= 0.604068021
Epoch: 0013 cost= 0.284048448
Epoch: 0017 cost= 0.164325977
Epoch: 0021 cost= 0.112331383
Epoch: 0025 cost= 0.081791018
Epoch: 0029 cost= 0.063280030
Epoch: 0033 cost= 0.051531953
Epoch: 0037 cost= 0.042417211
Epoch: 0041 cost= 0.035144885
Epoch: 0045 cost= 0.029989285
Epoch: 0049 cost= 0.026129655
Optimization Finished!
Accuracy: 0.924989


In [24]:
%pylab inline
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score,confusion_matrix,classification_report

lab=pd.DataFrame(Y_test).idxmax(1).values
cnf_matrix = confusion_matrix(lab,predd)
print(cnf_matrix )

Populating the interactive namespace from numpy and matplotlib


C:\Users\hp\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle', 'mean']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


[[10647   209]
 [  666   143]]


In [26]:
print( predd )

[0 0 0 ..., 0 0 1]


In [27]:
N=np.linspace(1,10,10)

In [28]:

print(min(lab),max(lab))

0 1


In [29]:
print(lab.shape,predd.shape)

(11665,) (11665,)


In [30]:
A=tf.contrib.metrics.confusion_matrix(lab,predd) 

In [31]:
from sklearn.metrics import confusion_matrix


cnf_matrix = confusion_matrix(lab,predd)
np.set_printoptions(precision=2)
print(cnf_matrix )

[[10647   209]
 [  666   143]]
